In [7]:
import pandas as pd
import os
os.chdir("./data")
income_by_branch = pd.read_csv("cif-active-customer---income-range-by-branch.csv")
employment_state_occupation = pd.read_csv("bptms_Employed_persons_by_occupation_state.csv")
compensation_economic_activity = pd.read_csv(
    "bpan_2_Compensation_of_Employees_by_Kind_of_Economic_Activity_at_Current_Prices_RM_Million.csv")
household_district = pd.read_csv(
    "bppd_Household_and_Living_Quarters_by_Administrative_District_and_State_Malaysia_2010.csv")
population_district = pd.read_csv("bppd_Population_by_Sex_Administrative_District_and_State_Malaysia_2010.csv")

# What do we want to achieve?
We want to be able to identify the underserved community, for instance poor neighbourhood. 

# How do we want to achieve that?
We understand that the demographics is changing and that demographic data in Malaysia is limited. At the same time, we also want to ensure that the voices of the minority are not drown out by the majority. Therefore we want to create a model which can be used as a proxy of income in neighbourhood and estimate the percentage of minority, so that their votes are given more weightage.

# Example of how it will work
[Input] --**Issue** --> tag whether involving underserved community --> tag the related underserved community --> less votes needed to increase in priority [Output]

[Input] --**Person data** --> tag community --> Increase voting weightage for specific issue related to the belonging communities [Output]  
# Data used
1. Malaysian population by district
2. Malaysian household by district
3. Salary by economic activity
4. Employment by state and occupation
5. Income by branches in a bank

# Desired characteristics of model
1. Explainable - to show transparency and if flawed reason, can be flagged and resolved manually
2. Can work when there are empty columns (certain data may not exist for district)

# Possible models
- rule-based
- trees

In [10]:
income_by_branch.columns = [a.strip() for a in income_by_branch.columns]
income_by_branch = income_by_branch.drop(columns = ['BRNO', 'Grand Total', 'NA'])
income_by_branch = income_by_branch.fillna(0)
income_by_branch.head(10)

,BRANCH NAME,STATE,"RM0-RM1,500",">RM1,501-RM2,500",">RM2,500-RM5,000",">RM5,000-RM8,000",">RM8,000-RM15,000",">RM15,000-RM20,000",">RM20,000-RM30,000",">RM30,000-RM50,000",">RM50,0000"
0,Kangar,PERLIS,"29,196","4,425","5,560","1,462",291,6,13,5,7.0
1,Alor Setar,KEDAH,"22,804","4,568","5,773","1,483",297,18,6,12,7.0
2,Sungai Petani,KEDAH,"30,677","4,170","6,484","1,997",359,26,26,14,5.0
3,Baling,KEDAH,"22,514","2,267","2,950",721,63,3,4,4,1.0
4,Bagan Serai,PERAK,"14,684","2,797","3,448",916,93,9,4,0,1.0
5,Sungai Besar,SELANGOR,"16,522","2,799","3,091",834,101,1,4,3,2.0
6,Jalan Ipoh,W.P KUALA LUMPUR,"18,439","6,595","6,257","1,232",471,76,54,23,16.0
7,Melaka,MELAKA,"32,314","5,184","5,821","1,296",333,42,40,11,9.0
8,Muar,JOHOR,"19,313","3,758","5,381","1,197",222,12,6,1,3.0
9,Temerloh,PAHANG,"18,496","2,998","3,954",990,151,12,6,3,1.0


In [50]:
employment_state_occupation = employment_state_occupation.drop(columns = ['Code Version'])
print(employment_state_occupation.loc[17:19,:])
employment_state_occupation['Category of occupation'] = [a.strip() for a in employment_state_occupation['Category of occupation']]
print("\n\nCategories:\n", employment_state_occupation['Category of occupation'].unique())

   State/Country  Year Category of occupation Employed Person ('000)
17      Malaysia  2016               Managers                  658.5
18         Johor  2016               Managers                   61.6
19         Kedah  2016               Managers                   26.3


Categories:
 ['Total' 'Managers' 'Professionals'
 'Technicians and associate professionals' 'Clerical support workers'
 'Service and sales workers'
 'Skilled agricultural, forestry,livestock and fishery workers'
 'Craft and related trades workers'
 'Plant and machine-operators and assemblers' 'Elementary occupations'
 'Skilled agricultural, forestry and fishery workers'
 'Legislators, senior officials and managers' 'Clerical workers'
 'Service workers and shop and market sales workers'
 'Skilled agricultural and fishery workers'
 'Craft and related trade workers'
 'Professional, technical and related workers'
 'Administrative and managerial workers' 'Clerical and related workers'
 'Sales workers' 'Service workers

In [26]:
compensation_economic_activity.head(3)

,Year,Agriculture,Mining and quarrying,Manufacturing,Construction,Services,Total
0,2005,8219,2610,48912,11268,89586,160594
1,2006,9451,2871,53057,11774,100773,177926
2,2007,12301,3760,56402,13859,115454,201775


In [30]:
household_district.head(3)

,Year,State,Administrative District,Households,Living Quarters
0,2010,Johor,Batu Pahat,"91,198","108,065"
1,2010,Johor,Johor Bahru,"331,221","389,145"
2,2010,Johor,Kluang,"70,280","80,525"


In [32]:
population_district.head(3)

,Year,State,Administrative District,Total,Male,Female
0,2010,Johor,Batu Pahat,"401,902","206,220","195,682"
1,2010,Johor,Johor Bahru,"1,334,188","708,660","625,528"
2,2010,Johor,Kluang,"288,364","160,007","128,357"


In [37]:
composite_table = pd.merge(household_district, 
                           population_district,  how='outer', 
                           left_on=['Year', 'State','Administrative District'],
                           right_on = ['Year', 'State','Administrative District'])
composite_table.head(3)

,Year,State,Administrative District,Households,Living Quarters,Total,Male,Female
0,2010,Johor,Batu Pahat,"91,198","108,065","401,902","206,220","195,682"
1,2010,Johor,Johor Bahru,"331,221","389,145",NaN,NaN,NaN
2,2010,Johor,Kluang,"70,280","80,525",NaN,NaN,NaN


**We can see already how some of the data is unavailable**
This is a constraint that the models need to be able to tolerate. 

# ya-da ya-da ya-da 
Voila the final data is like: